# Projeto - Crédito para Financiamento de Imóveis

## Avaliação do módulo de Data Engineering

### Base de Dados

Serão utilizadas bases de dados com informações cadastrais, histórico de crédito e balanços financeiros de diversos clientes. O conjunto de dados está dividido em treino e teste, todos no formato csv. Toda a modelagem, validação e avaliação deve ser feita em cima do conjunto de treino, subdividindo tal base como achar melhor. Existe também a base das variáveis explicativas, para ajudar no desenvolvimento do projeto.

[Baixar aqui](https://s3-sa-east-1.amazonaws.com/lcpi/0694c90a-7782-47f7-8bbc-e611d31f9f21.zip)

### Contextualização

A PyCoders Ltda., cada vez mais especializada no mundo da Inteligência Artificial e Ciência de Dados, foi procurada por uma fintech para desenvolver um projeto de concessão de crédito para imóveis. Nesse projeto, espera-se a criação de score que discrimine ao máximo os bons pagadores dos maus pagadores. Para isso, foi disponibilizada uma base de dados com milhares de casos de empréstimos do passado, com diversas características dos clientes. Deve ser entregue um modelo para realizar essa classificação. Por questões contratuais, o pagamento será realizado baseado no desempenho (ROC AUC).





### Antes de modelar

1. Crie um `fluxo de dados` no Databricks para as bases que serão utiizadas.
    1. Insira os dados brutos na primeira camada.
    1. Salve as transformações / limpezas na segunda camada.
   
    1. Crie uma pipeline para o processo.
        1. Pipeline, nesse caso, será um único script que gere todas as tabelas acima.

### Durante a modelagem

1. Selecione e salve as colunas relevantes e features criadas na terceira camada.

1. Salve as versões do modelo no formato `pickle` no DBFS.

1. Mantenha o controle das versões criando uma tabela no formato
**ATENÇÂO:** Como estamos na versão community, lembre de exportar a tabela abaixo para o DBFS antes da sessão encerrar.

|id_modelo|nome_modelo|data_treino|método|roc_auc|tempo_de_treino(s)|hyperparametros|path_to_pickle
|---|---|---|---|---|---|---|---|
|1|RandomForestRapida|2022-02-22|Random Forest Simples|0.76|124|\[max_depth=4, ...\]|/FileStore/tables/modelos/...|

### Após modelar

1. Salve os dados de treino e validação em uma tabela (para que seja possível reproduzir resultados no futuro).
    1. Crie uma coluna com o id do modelo escolhido para refêrencia. 

### Regras de Entrega

1. Um notebook (databricks) com a pipeline que gere as tabelas da fase *Antes de modelar*. O notebook deve rodar de uma vez só!

1. Um arquivo `.csv` com as informações da tabela gerada na fase *Durante a modelagem*.

1. Um notebook que gere as tabelas de treino e validação do passo *Após modelar*. O notebook deve rodar de uma vez só!

> **IMPORTANTE:** Tendo em vista que não teremos apresentação do projeto, é indispensável que ele esteja organizado e comentado.

# Resolução

## Recursos auxiliares para visualização dos dados

> Para facilitar o processo de análise dos dados e escolha do tipo de limpeza/transformações que adotaremos, resolvemos criar algumas funções auxiliares para utilizar a [tabela de descrição das colunas](../data/HomeCredit_columns_description.csv) de modo mais interativo e dinânmico. 

In [3]:
# Importação das bibliotecas

import pandas as pd

In [9]:
# Criação de um dataframe com a base de treino

df_train = pd.read_csv('../data/application_train.csv')
df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0


In [4]:
# Criação de um dataframe com a tabela de descrição das colunas

df_cols = pd.read_csv('../data/HomeCredit_columns_description.csv', encoding= 'unicode_escape', index_col=0)
df_cols = df_cols[df_cols['Table'] == 'application_{train|test}.csv']
df_cols = df_cols[['Row', 'Description']]
df_cols.head()

,Row,Description
1,SK_ID_CURR,ID of loan in our sample
2,TARGET,Target variable (1 - client with payment diffi...
5,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving
6,CODE_GENDER,Gender of the client
7,FLAG_OWN_CAR,Flag if the client owns a car


In [5]:
# Função auxiliar 1

def search_definition(dataframe):
    column_name = input('Digite o nome da coluna (0 para sair): ')
    definition = dataframe[dataframe['Row'] == column_name]['Description']
    print(definition.values[0])

In [6]:
search_definition(df_cols)

ID of loan in our sample


In [11]:
# Função auxiliar 2

def print_all_definitions(dataframe_full, dataframe_cols):
	for column in dataframe_full.columns:
		print("COLUMN:", column)
		print(dataframe_cols[dataframe_cols['Row'] == column]['Description'].values[0])
		print("\n")

In [12]:
print_all_definitions(df_train, df_cols)

COLUMN: SK_ID_CURR
ID of loan in our sample


COLUMN: TARGET
Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)


COLUMN: NAME_CONTRACT_TYPE
Identification if loan is cash or revolving


COLUMN: CODE_GENDER
Gender of the client


COLUMN: FLAG_OWN_CAR
Flag if the client owns a car


COLUMN: FLAG_OWN_REALTY
Flag if client owns a house or flat


COLUMN: CNT_CHILDREN
Number of children the client has


COLUMN: AMT_INCOME_TOTAL
Income of the client


COLUMN: AMT_CREDIT
Credit amount of the loan


COLUMN: AMT_ANNUITY
Loan annuity


COLUMN: AMT_GOODS_PRICE
For consumer loans it is the price of the goods for which the loan is given


COLUMN: NAME_TYPE_SUITE
Who was accompanying client when he was applying for the loan


COLUMN: NAME_INCOME_TYPE
Clients income type (businessman, working, maternity leave,)


COLUMN: NAME_EDUCATION_TYPE
Level of highest educati

## Documentação do processo de resolução

> Upload das tabelas no `/FileStore/tables/projeto_credito/` do Databricks:
> 
> ![Upload das tabelas](../imgs/01.jpg)
> 
> Link para notebook do Databricks com a pipeline da etapa `Antes de Modelar` [aqui](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3711839099470423/2262021360226139/5022278466657041/latest.html).